In [52]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

## Load all data

In [46]:
meta = pd.read_csv("data/meta.csv")


In [47]:
valid_data = meta.loc[meta["edges"]>0,:]

In [48]:
def get_modularity_from_edgelist(edge_df:pd.DataFrame) -> float:
    G = nx.from_pandas_edgelist(edge_df,"from","to")
    return nx.community.modularity(G,nx.community.louvain_communities(G,seed=420))

In [49]:
modularity_values = []
for id in tqdm(valid_data["id"]):
    #print(id)
    temp_edgelist = pd.read_csv("data/{}/edges.csv".format(id))
    modularity_values.append(get_modularity_from_edgelist(temp_edgelist))



100%|██████████| 133/133 [00:11<00:00, 11.94it/s]


In [ ]:
valid_data["modularity"] = modularity_values